In [1]:
#@title dop.func
# Разовое изменение шрифта
import time

from IPython.display import Javascript
def set_font1():
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '16px'
      break
    }
  }
  '''))


# Разовое сокращение 
# from IPython.display import Javascript
def frame_height():
    display(Javascript('''
    google.colab.output.setIframeHeight(0, true, {maxHeight: 550})
    '''))

print(*time.localtime()[0:3], sep='-')
print('✅ Дополнительные функции')

2023-2-26
✅ Дополнительные функции


# **♻️ LOTTO**

In [2]:
from random import randint as rnd
import os

"""
def clear_screen():
    # Проверка: Operating System == Mac & Linux || Windows
    if os.name == 'posix':
        _ = os.system('clear')
    else:
        # Иначе: Operating System = Windows (os.name = nt)
        _ = os.system('cls')
"""

from IPython.display import clear_output
def clear_screen():
    clear_output()


def random_list(nums):
    r_list = []
    while len(r_list) < nums:
        rnd_num = rnd(1, 90)
        if rnd_num not in r_list:
            r_list.append(rnd_num)
    return r_list


def emoji_digits(num, emoji='⬜️'): # Функция замены чисел значками emoji
    digits = ['0️⃣', '1️⃣', '2️⃣', '3️⃣', '4️⃣', '5️⃣', '6️⃣', '7️⃣', '8️⃣', '9️⃣']
    if num < 10:
        emoji_digits = f'{emoji}{str(digits[num])}' # f'⬜️{str(digits[num])}'
    else:
        digit1 = int(str(num)[0])
        digit2 = int(str(num)[1])
        emoji_digits = f'{str(digits[digit1])}{str(digits[digit2])}'
    return emoji_digits

In [3]:
from random import randint as rnd

#from tools import generate_unique_numbers
class Card:
    card_rows = 3
    card_cols = 9
    card_nums_in_row = 5
    card_data = None
    card_empty_num = 0
    card_crossed_num = -1
    emj = '⬜️'

    def __init__(self, i_user):
        uniques_num = self.card_nums_in_row * self.card_rows # количество уникальных чисел 5*3=15
        uniques = random_list(uniques_num)
        self.emj = Game.card_colors[i_user]

        self.card_data = [] #  формирование карточки 
        for i in range(0, self.card_rows): # перебор 3 рядов (от 0 до 2)
            tmp = sorted(uniques[self.card_nums_in_row * i: self.card_nums_in_row * (i + 1)]) # списки [0:5][5:10][10:15]
            empty_nums_count = self.card_cols - self.card_nums_in_row  # 9-5=4
            for j in range(0, empty_nums_count): # заполняем карточку пробелами (0) на случайных позициях
                index = rnd(0, len(tmp))
                tmp.insert(index, self.card_empty_num)
            self.card_data += tmp   # формируем полную карточку для игры


    def __str__(self):  # Функция заполнения карточки с числами графичискими элементами
        emj0 = '🔲'
        str_line = '〰️' * 26
        result = str_line + '\n'
        for index, num in enumerate(self.card_data):
            if num == self.card_empty_num:      # заполнение пробелов (=0)
                result +=  self.emj * 2     # '⬜️⬜️'
            elif num == self.card_crossed_num:  # вместо зачеркнутых чисел
                result += self.emj + emj0   # '⬜️🔲'
            else:
                result += emoji_digits(num, self.emj)

            if (index + 1) % self.card_cols == 0:  # Перевод строки за последним числом в ряду
                result += '\n'
            else:
                result += self.emj          # '⬜️'
        return result + str_line


    def __contains__(self, item): # метод класса: содержится ли заданный элемент в карточке игрока
        is_contains = item in self.card_data
        return is_contains


    def cross_num(self, num): # функция "зачеркивания" чисел на карточке при совпадении
        for index, item in enumerate(self.card_data):
            if item == num:
                self.card_data[index] = self.card_crossed_num
                return
        raise ValueError(f'❌ На карточке нет числа: {num}')


    def closed(self) -> bool:  # если множества равны, то все ячейки на карточке заполнены, и функция возвращает True
        is_closed = set(self.card_data) == {self.card_empty_num, self.card_crossed_num}
        return is_closed

In [4]:
import time
class Game:
    player_cards = []
    game_barrels = []
    card_colors = ['⬜️', '🟥', '🟨', '🟦', '🟧', '🟪', '🟫', '🟩']
    user_icons = ['🤖 RO-Bot', '🚗', '🚕', '🚙', '🚚', '🚑', '🚒', '🚜']

    def __init__(self, gamers_count):
        gamers_max = 2 if gamers_count < 3 else gamers_count
        for i in range(gamers_max):
            self.player_cards.append(Card(i+1))
        self.game_barrels = random_list(90)

    def ask_to_cross(self, game_user_card, barrel) -> bool:
        # При выборе "зачеркнуть": если цифры на карточке нет - игрок проигрывает, и игра завершается
        # При выборе "продолжить": если цифра есть на карточке - игрок проигрывает, и игра завершается
        # ... иначе - игра продолжается!
        y_n = input('... зачеркнуть [y] ... нет, продолжить [n]: ').lower().strip()
        if y_n == 'y' and not barrel in game_user_card or y_n != 'y' and barrel in game_user_card:
            return False   
        return True   


    def check_winner_card(self, game_user_card, barrel):
        if barrel in game_user_card:          # зачеркиваем число в карточке текущего игрока (при наличии)
            game_user_card.cross_num(barrel)
            if game_user_card.closed():       # если карточка текущего игрока полностью закрыта, то победа!
                return True                   # объявляем победителя
        return False                          # продолжаем игру


    def play_round(self):
        # функция выдает комбинацию status и номер user (int)
        # user=-1 с проигравшим или победителем не определились: продолжаем игру
        # если False, то игра прекращается и возвращаем номер проигравшего
        # если True, то игра прекращается и возвращаем номер победителя
        status = False # статус: False=мгновенный проигрыш или True=выигрыш конкретного игрока
        barrel = self.game_barrels.pop()
        emoji_barrel = emoji_digits(barrel)
        print(f'\n🎲 Фишка №: {emoji_barrel}  [{90-len(self.game_barrels)}/{len(self.game_barrels)}]')
        for i, card in enumerate(self.player_cards):
            if gamers_count == 0 or (gamers_count == 1 and i == 0): # Для игр: Бот <-> Бот или Бот <-> Человек
                print(f'\n{self.card_colors[i+1]} Карточка {self.user_icons[0]}-{i+1}\n{card}')
                time.sleep(0.5)                                     # для Бота вопрос "что делать?" опускаем - пауза для наглядности
            else:                                                   #
                print(f'\n{self.card_colors[i+1]} Карточка игрока {i+1} [{self.user_icons[i+1]}]\n{card}')
                status = self.ask_to_cross(card, barrel)            # для игроков запрашиваем, что делать: зачеркнуть [y], иначе - дальше
                if not status:                                      # если игрок ошибся (False), то прекращаем игру 
                    return status, i+1                              # 🆘 возвращаем номер проигравшего игрока (поэтому номер на 1 больше)
            status = self.check_winner_card(card, barrel)           # проверяем остаток незачеркнутых чисел в карточке текущего Бота/игрока
            if status:                                              # если на карточке не осталось чисел, то - окончание игры
                return status, i+1                                  # ✅ при заполнненой карточке объявляем победителя
        
        time.sleep(0.2)
        clear_screen()
        return status, -1                                          # [-1] с проигравшим или победителем не определились: продолжаем игру

In [5]:
# from games import Game
info = """
    Варианты игры:
    0    компьютер <-> компьютер
    1    человек <-> компьютер
    2    человек <-> человек
    3-7  От 3 и до 7 человек между собой
"""

if __name__ == '__main__':
    print(f'\n{info}')
    try:
        gamers_count = int(input(f'\n💠 Для выбора игры выбрать число от 0 до 7: '))
        if gamers_count not in range(0,8):
            raise ValueError('⛔️ Ошибка выбора игры!')
        lotto_game = Game(gamers_count)
    except ValueError as error:
        print(error)
        

    while True:
        status, result = lotto_game.play_round() # Возвращается False (мгновенный проигрыш) или True (выигрыш) + номер Игрока
        if result != -1:  # для продолжения игры возвращается -1, иначе индекс победившего игрока (0=Bot)
            w = '〰️' * 16
            if gamers_count > 0:
                user = '🤖 RO-Bot-1 !' if result == 0 else f'Игрок {result} {Game.user_icons[result]}'
            else: # gamers_count == 0
                user = f'🤖 RO-Bot {result}!'
            if status:
                print(f'\n{w}\n🥇🥇🥇 Победил {user} 🎉🎉🎉\n{w}')
            else:
                print(f'\n{w}\n🆘 {user} проиграл ...\n{w}')
            break # останов игры 


🎲 Фишка №: ⬜️1️⃣  [89/1]

🟥 Карточка 🤖 RO-Bot-1
〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️
🟥🟥🟥🟥🔲🟥🟥🔲🟥🟥🔲🟥🟥🔲🟥🟥🟥🟥🟥🟥🟥🟥🔲🟥🟥🟥
🟥1️⃣🟥🟥🔲🟥🟥🟥🟥🟥🔲🟥🟥🟥🟥🟥🟥🟥🟥🔲🟥🟥🔲🟥🟥🟥
🟥🔲🟥🟥🔲🟥🟥🔲🟥🟥🟥🟥🟥🔲🟥🟥🟥🟥🟥🟥🟥🟥🔲🟥🟥🟥
〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️

〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️
🥇🥇🥇 Победил 🤖 RO_Bot 1! 🎉🎉🎉
〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️〰️
